In [32]:
!pip -q install pandas

import os, re, json
import pandas as pd

METADATA_JSON = "ontology_ready_metadata.json"
DOCS_CSV      = "generated_definitions.csv"

OUTPUT_DIR = "judge_reports"
os.makedirs(OUTPUT_DIR, exist_ok=True)

!pip -q install rdflib

from rdflib import Graph, RDF, RDFS, OWL
import re

load inputs

In [33]:
with open(METADATA_JSON, "r", encoding="utf-8") as f:
    metadata_raw = json.load(f)

docs_df = pd.read_csv(DOCS_CSV)


    Returns a DataFrame with at least:
    table_name, column_name, data_type, is_pk, is_fk, fk_table, fk_column, canonical_id
    Handles common shapes:
      A) {"tables":[{"table_name":..., "columns":[...]}]}
      B) list of dict rows [{table_name, column_name, ...}, ...]


Threshold used: coverage => 95%, weak_def_ration <= 20%, missing >0

In [34]:
def normalize_metadata(metadata_raw):
    rows = []

    if isinstance(metadata_raw, dict) and "tables" in metadata_raw:
        for t in metadata_raw["tables"]:
            tname = t.get("table_name") or t.get("name") or t.get("table")
            cols = t.get("columns", [])
            for c in cols:
                rows.append({
                    "table_name": tname,
                    "column_name": c.get("column_name") or c.get("name") or c.get("column"),
                    "data_type": c.get("data_type") or c.get("type"),
                    "is_pk": bool(c.get("is_pk") or c.get("pk") or c.get("primary_key")),
                    "is_fk": bool(c.get("is_fk") or c.get("fk") or c.get("foreign_key")),
                    "fk_table": c.get("fk_table") or c.get("ref_table"),
                    "fk_column": c.get("fk_column") or c.get("ref_column"),
                    "canonical_id": c.get("canonical_id") or c.get("id")
                })

    elif isinstance(metadata_raw, list):
        for c in metadata_raw:
            rows.append({
                "table_name": c.get("table_name") or c.get("table"),
                "column_name": c.get("column_name") or c.get("name") or c.get("column"),
                "data_type": c.get("data_type") or c.get("type"),
                "is_pk": bool(c.get("is_pk") or c.get("pk") or c.get("primary_key")),
                "is_fk": bool(c.get("is_fk") or c.get("fk") or c.get("foreign_key")),
                "fk_table": c.get("fk_table") or c.get("ref_table"),
                "fk_column": c.get("fk_column") or c.get("ref_column"),
                "canonical_id": c.get("canonical_id") or c.get("id")
            })
    else:
        raise ValueError("Unknown metadata format. Open the JSON and confirm its top-level structure.")

    df = pd.DataFrame(rows)
    df["table_name"] = df["table_name"].astype(str)
    df["column_name"] = df["column_name"].astype(str)
    return df


def guess_docs_columns(df: pd.DataFrame):
    cols = {c.lower(): c for c in df.columns}
    table_col = None
    col_col = None
    def_col = None

    # common exact matches
    for k in cols:
        if k in ["table", "table_name", "tablename"]:
            table_col = cols[k]
        if k in ["column", "column_name", "colname", "field", "field_name"]:
            col_col = cols[k]
        if k in ["definition", "description", "generated_definition", "column_definition"]:
            def_col = cols[k]

    # fallback heuristics
    if table_col is None:
        for c in df.columns:
            if "table" in c.lower():
                table_col = c; break
    if col_col is None:
        for c in df.columns:
            if "column" in c.lower() or "field" in c.lower():
                col_col = c; break
    if def_col is None:
        for c in df.columns:
            if "def" in c.lower() or "desc" in c.lower():
                def_col = c; break

    if not (table_col and col_col and def_col):
        raise ValueError(f"Could not infer docs columns. df.columns={list(df.columns)}")

    return table_col, col_col, def_col


def is_weak_definition(def_text, col_name):
    if pd.isna(def_text):
        return True
    s = str(def_text).strip()
    if len(s) < 15:
        return True

    # normalize
    norm = re.sub(r"[^a-z0-9]+", " ", s.lower()).strip()
    col_norm = re.sub(r"[^a-z0-9]+", " ", str(col_name).lower()).strip()

    # definition mostly repeats column name
    if col_norm and col_norm in norm and len(norm) <= len(col_norm) + 12:
        return True

    # generic filler
    generic = ["stores information", "contains information", "this column", "tbd", "unknown", "n/a"]
    if any(g in norm for g in generic):
        return True

    return False


def judge_docs_for_table(meta_df, docs_df_norm, table_name):
    m = meta_df[meta_df["table_name"] == table_name]
    d = docs_df_norm[docs_df_norm["table_name"] == table_name]

    meta_cols = set(m["column_name"].tolist())
    doc_cols  = set(d["column_name"].tolist())

    missing = sorted(list(meta_cols - doc_cols))
    extra   = sorted(list(doc_cols - meta_cols))

    weak = []
    for _, row in d.iterrows():
        if is_weak_definition(row["definition"], row["column_name"]):
            weak.append({
                "column": row["column_name"],
                "definition": row["definition"],
                "reason": "empty/short/tautology/generic"
            })

    coverage = 1.0 if len(meta_cols) == 0 else (len(meta_cols) - len(missing)) / len(meta_cols)
    weak_ratio = 0.0 if len(d) == 0 else len(weak) / len(d)

    # PASS/FAIL thresholds (change as needed)
    issues = []
    if coverage < 0.95:
        issues.append(f"Doc coverage too low: {coverage:.2%} (need >= 95%)")
    if len(missing) > 0:
        issues.append(f"Missing definitions for {len(missing)} columns")
    if weak_ratio > 0.20:
        issues.append(f"Too many weak definitions: {len(weak)}/{len(d)} (~{weak_ratio:.1%})")

    decision = "PASS" if len(issues) == 0 else "FAIL"

    report = {
        "table": table_name,
        "decision": decision,
        "docs_metrics": {
            "schema_column_count": int(len(meta_cols)),
            "doc_row_count": int(len(d)),
            "coverage": coverage,
            "missing_definition_count": int(len(missing)),
            "extra_definition_count": int(len(extra)),
            "weak_definition_count": int(len(weak)),
            "weak_definition_ratio": weak_ratio,
        },
        "docs_details": {
            "missing_definitions": missing,
            "extra_definitions": extra,
            "weak_definitions_sample": weak[:25],  # cap
        },
        "issues": issues
    }

    return report


def build_fix_list(report):
    fixes = []
    t = report["table"]

    for c in report["docs_details"]["missing_definitions"][:50]:
        fixes.append({
            "table": t,
            "type": "DOC_MISSING",
            "column": c,
            "fix": f"Add a plain-English definition for {t}.{c} (meaning + business context + units/allowed values if relevant)."
        })

    for item in report["docs_details"]["weak_definitions_sample"][:50]:
        fixes.append({
            "table": t,
            "type": "DOC_WEAK",
            "column": item["column"],
            "fix": f"Rewrite definition for {t}.{item['column']} to avoid tautology/generic text. Include what it represents + any constraints/units."
        })

    return fixes

In [40]:
meta_df = normalize_metadata(metadata_raw)

DOC_TABLE_COL, DOC_COL_COL, DOC_DEF_COL = guess_docs_columns(docs_df)
docs_df_norm = docs_df[[DOC_TABLE_COL, DOC_COL_COL, DOC_DEF_COL]].copy()
docs_df_norm.columns = ["table_name", "column_name", "definition"]
docs_df_norm["table_name"] = docs_df_norm["table_name"].astype(str)
docs_df_norm["column_name"] = docs_df_norm["column_name"].astype(str)

# Run Evaluation for ALL tables
all_tables = sorted(meta_df["table_name"].unique().tolist())

reports = []
fixes_all = []

# Precompute ontology block once (global delta vs core)
onto_block = ontology_eval_block()

# if ontology duplication should force FAIL globally?
dup_class_count = onto_block["duplicates_vs_core"]["classes_dup_by_localname_count"]

for t in all_tables:
    r = judge_docs_for_table(meta_df, docs_df_norm, t)

    # Attach ontology results before saving
    r["ontology"] = onto_block

    # If you want strict governance: duplicates => FAIL
    if dup_class_count > 0:
        r["issues"].append(
            f"Delta ontology duplicates core classes by local name: {dup_class_count} (must reuse core classes before merge)."
        )
        r["decision"] = "FAIL"

    reports.append(r)

    # per-table report
    safe_t = re.sub(r"[^A-Za-z0-9_]+", "_", t)
    with open(os.path.join(OUTPUT_DIR, f"judge_report_{safe_t}.json"), "w", encoding="utf-8") as f:
        json.dump(r, f, indent=2)

    if r["decision"] == "FAIL":
        fixes_all.extend(build_fix_list(r))

Summary

In [41]:
summary_rows = []
for r in reports:
    m = r["docs_metrics"]
    summary_rows.append({
        "table": r["table"],
        "decision": r["decision"],
        "coverage": m["coverage"],
        "schema_column_count": m["schema_column_count"],
        "doc_row_count": m["doc_row_count"],
        "missing_definition_count": m["missing_definition_count"],
        "weak_definition_count": m["weak_definition_count"],
        "weak_definition_ratio": m["weak_definition_ratio"],
        "issue_count": len(r["issues"])
        ,"thresholds": {
    "min_doc_coverage": 0.95,
    "max_weak_definition_ratio": 0.20,
    "fail_on_duplicate_class_localname": True
}
    })

summary_df = pd.DataFrame(summary_rows).sort_values(["decision", "coverage", "missing_definition_count"])
summary_df.to_csv("judge_summary.csv", index=False)

with open("fix_list_all.json", "w", encoding="utf-8") as f:
    json.dump(fixes_all, f, indent=2)

print("Done.")
print("Reports written to:", OUTPUT_DIR)
print("Summary written to: judge_summary.csv")
print("Fix list written to: fix_list_all.json")
summary_df.head(15)

Done.
Reports written to: judge_reports
Summary written to: judge_summary.csv
Fix list written to: fix_list_all.json


,table,decision,coverage,schema_column_count,doc_row_count,missing_definition_count,weak_definition_count,weak_definition_ratio,issue_count,thresholds
0,foundation_account_authgateway_session_created...,PASS,1.0,5,5,0,0,0.0,0,"{'min_doc_coverage': 0.95, 'max_weak_definitio..."
1,foundation_account_verification_attempt,PASS,1.0,5,5,0,0,0.0,0,"{'min_doc_coverage': 0.95, 'max_weak_definitio..."
2,foundation_account_verified_user_details,PASS,1.0,5,5,0,0,0.0,0,"{'min_doc_coverage': 0.95, 'max_weak_definitio..."
3,foundation_financial_deposits,PASS,1.0,7,10,0,0,0.0,0,"{'min_doc_coverage': 0.95, 'max_weak_definitio..."
4,foundation_financial_ledger_account_balances,PASS,1.0,5,5,0,0,0.0,0,"{'min_doc_coverage': 0.95, 'max_weak_definitio..."
5,foundation_financial_ledger_lines,PASS,1.0,5,5,0,0,0.0,0,"{'min_doc_coverage': 0.95, 'max_weak_definitio..."
6,foundation_financial_withdrawals,PASS,1.0,7,10,0,0,0.0,0,"{'min_doc_coverage': 0.95, 'max_weak_definitio..."
7,foundation_financial_worldpay_transactions,PASS,1.0,5,5,0,0,0.0,0,"{'min_doc_coverage': 0.95, 'max_weak_definitio..."


Load OWL's

In [42]:
DELTA_OWL = "final_grounded_ontology.owl"
CORE_OWL  = "data_democratization_ontology.owl"

delta_g = Graph()
delta_g.parse(DELTA_OWL)

core_g = Graph()
core_g.parse(CORE_OWL)

def local_name(uri):
    s = str(uri)
    if "#" in s:
        return s.rsplit("#", 1)[-1]
    return s.rsplit("/", 1)[-1]

def extract_signature(g: Graph):
    classes = set(g.subjects(RDF.type, OWL.Class))
    obj_props = set(g.subjects(RDF.type, OWL.ObjectProperty))
    dt_props  = set(g.subjects(RDF.type, OWL.DatatypeProperty))
    return {
        "classes": classes,
        "object_properties": obj_props,
        "datatype_properties": dt_props
    }

delta_sig = extract_signature(delta_g)
core_sig  = extract_signature(core_g)

def dup_report(delta_set, core_set):
    core_uris = set(core_set)
    core_local = {}
    for x in core_set:
        core_local.setdefault(local_name(x).lower(), []).append(x)

    dup_by_uri = sorted([str(x) for x in delta_set if x in core_uris])

    dup_by_local = []
    for x in delta_set:
        ln = local_name(x).lower()
        if ln in core_local and all(x != y for y in core_local[ln]):
            for y in core_local[ln]:
                dup_by_local.append((str(x), str(y)))

    # de-dup pairs
    seen = set()
    out = []
    for a,b in dup_by_local:
        key = (a,b)
        if key not in seen:
            seen.add(key)
            out.append(key)

    return {"dup_by_uri": dup_by_uri, "dup_by_localname": out}

dup_classes = dup_report(delta_sig["classes"], core_sig["classes"])
dup_objprops = dup_report(delta_sig["object_properties"], core_sig["object_properties"])
dup_dtprops = dup_report(delta_sig["datatype_properties"], core_sig["datatype_properties"])

print("Loaded OWLs:",
      "\n- delta triples:", len(delta_g),
      "\n- core triples:", len(core_g),
      "\nDup classes(localname):", len(dup_classes["dup_by_localname"]),
      "\nDup object props(localname):", len(dup_objprops["dup_by_localname"]),
      "\nDup datatype props(localname):", len(dup_dtprops["dup_by_localname"]))


Why:

* The score for WorldpayTransaction is 0.95, which is significantly higher than the other candidates.
* The label "None" suggests that this candidate has a specific context related to financial transactions in the Worldpay domain.

By choosing WorldpayTransaction as the best candidate, we can assume that the foundation.financial.worldpay_transactions asset is likely referring to a collection of financial transaction data specific to Worldpay, which aligns with its high score and specific label. does not look like a valid URI, trying to serialize this will break.

This is because VerifiedUserProfile has the highest score (0.7), indicating that it is the most relevant and accurate candidate for the technical asset "foundation.account.verified_user_details". The other candidates have lower scores, suggesting that they may not be as well-suited for this specific task. does not look like a valid URI, trying to serialize this will break.

This is because it has the highest score of 0.

Loaded OWLs: 
- delta triples: 879 
- core triples: 738 
Dup classes(localname): 0 
Dup object props(localname): 0 
Dup datatype props(localname): 0


Simple "sanity checks"

In [38]:
def ontology_sanity_checks(delta_sig):
    issues = []

    # "*_id" should usually NOT be an ObjectProperty
    for p in delta_sig["object_properties"]:
        ln = local_name(p).lower()
        if ln.endswith("_id") or ln.endswith("id"):
            issues.append({
                "type": "ONTO_SANITY",
                "rule": "ID-like name as ObjectProperty",
                "item": str(p),
                "why": "IDs are typically literals (datatype properties). Relationships should be modeled as object properties separately."
            })

    # timestamp/date columns shouldn't be ObjectProperty
    for p in delta_sig["object_properties"]:
        ln = local_name(p).lower()
        if any(k in ln for k in ["date", "time", "timestamp", "created", "updated"]):
            issues.append({
                "type": "ONTO_SANITY",
                "rule": "Time-like name as ObjectProperty",
                "item": str(p),
                "why": "Time/timestamp fields are typically literals (datatype properties)."
            })

    return issues

sanity_issues = ontology_sanity_checks(delta_sig)
print("Sanity warnings:", len(sanity_issues))

Sanity warnings: 0


added ontology results into table results

In [39]:
def ontology_eval_block():
    return {
        "delta_counts": {
            "classes": len(delta_sig["classes"]),
            "object_properties": len(delta_sig["object_properties"]),
            "datatype_properties": len(delta_sig["datatype_properties"]),
        },
        "duplicates_vs_core": {
            "classes_dup_by_uri_count": len(dup_classes["dup_by_uri"]),
            "classes_dup_by_localname_count": len(dup_classes["dup_by_localname"]),
            "object_props_dup_by_localname_count": len(dup_objprops["dup_by_localname"]),
            "datatype_props_dup_by_localname_count": len(dup_dtprops["dup_by_localname"]),
            # cap samples so reports stay readable
            "classes_dup_by_localname_sample": dup_classes["dup_by_localname"][:20],
            "object_props_dup_by_localname_sample": dup_objprops["dup_by_localname"][:20],
            "datatype_props_dup_by_localname_sample": dup_dtprops["dup_by_localname"][:20],
        },
        "sanity_warnings_count": len(sanity_issues),
        "sanity_warnings_sample": sanity_issues[:20]
    }